In [ ]:
from pyspider.libs.base_handler import *
import re
from datetime import datetime


class Handler(BaseHandler):
    crawl_config = {
    }

    retry_delay = {
        0: 0.1,
        1: 0.2,
        3: 0.3
    }
    
    @every(minutes=24 * 60)
    def on_start(self):
        self.crawl('http://lishi.tianqi.com/', callback=self.index_page)

    @config(age=10 * 24 * 60 * 60)
    def index_page(self, response):
        i = 1
        for each in response.doc('a[href^="http"]').items():
            if re.match("http://lishi.tianqi.com/.*/index.html", each.attr.href, re.U) :
                self.crawl(each.attr.href, callback=self.city_page)

       
    def city_page(self, response):
        city = response.url[24:-11]
        if city == '': 
            return
        date = datetime(2012, 1, 1)
        
        while date < datetime(2020, 1, 1) :     #选择日期，从2012到2019的每一个月
            date_str = date.isoformat()[:7].replace("-", "")
            
            self.crawl('http://lishi.tianqi.com/'+ city +'/'+ date_str +'.html', callback=self.detail_page)
            new_month = date.month + 1
            if new_month > 12:
                date = datetime(date.year + 1, 1, 1)
            else:
                date = datetime(date.year, new_month, 1)
                
    @config(priority=2)
    def detail_page(self, response):
        return {
            "url": response.url,
            "title": response.doc('title').text(),
            "head": response.doc('.thalin > .th200 , .thalin > .th140').text(),
            "date": response.doc('li > .th200').text(),
            "content": response.doc("li > .th140").text()
        }